## Data Loading

In [ ]:
import snowflake.connector
import streamlit as st
import pandas as pd
import numpy as np
# Connect to Snowflake

# Query data
query = "SELECT room_type, property_type, accommodates, bathrooms bedrooms, beds, price, minimum_nights, maximum_nights, availability_365, host_is_superhost, host_listings_count, instant_bookable, review_scores_rating, number_of_reviews, reviews_per_month, latitude, longtitude FROM LISTINGS_NYC;"
df = pd.read_sql(query, conn)

# Display the first few rows
print(df.head())

In [ ]:
df.info()

# 描述统计（数值型）
df.describe()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

## Data Cleaning

### Efficient basic processing

In [ ]:
-- SQL Cell
CREATE OR REPLACE TEMP TABLE cleaned_listings AS
SELECT
  id,
  TRY_TO_NUMBER(REPLACE(REPLACE(price, '$', ''), ',', '')) AS price,
  COALESCE(host_is_superhost, 'f') = 't' AS host_is_superhost,
  instant_bookable = 't' AS instant_bookable,
  COALESCE(host_listings_count, 0) AS host_listings_count,
  LN(1 + COALESCE(host_listings_count, 0)) AS host_listings_count_log,
  bedrooms,
  beds,
  accommodates,
  room_type,
  NEIGHBOURHOOD_GROUP_CLEANSED,
  property_type,
  minimum_nights,
  maximum_nights,
  availability_365,
  COALESCE(TRY_TO_NUMBER(reviews_per_month), 0) AS reviews_per_month,
  latitude,
  longtitude
FROM LISTINGS_NYC
WHERE minimum_nights <= 60 AND maximum_nights <= 365;


### Initial Data Cleaning Summary
We performed initial structured data cleaning in SQL to efficiently filter, convert, and preprocess fields close to the data source, reserving more complex imputations and modeling tasks for Python.

- `price`: Removed `$` and `,` characters and converted the result to a numeric value using `TRY_TO_NUMBER`.

- `host_is_superhost`: Replaced nulls with `'f'` (false) and converted `'t'/'f'` values to Boolean (`TRUE` if superhost).

- `instant_bookable`: Converted `'t'/'f'` values to Boolean for easier downstream processing.

- `host_listings_count`: Replaced missing values with `0` and created a log-transformed version `host_listings_count_log` to reduce skewness.

- `bedrooms` and `beds`: Left as-is for now. Missing values will be handled in Python using more advanced logic (e.g., based on nearby listings).

- `minimum_nights`, `maximum_nights`: Retained, but we filtered out extreme values by restricting to `≤ 60` and `≤ 365` respectively.

- `reviews_per_month`: Converted to numeric using `TRY_TO_NUMBER`, filled nulls with 0.

- All other features (`accommodates`, `room_type`, `property_type`, `number_of_reviews`, `latitude`, `longtitude`) were kept as-is for downstream feature engineering and modeling.

This table serves as the cleaned base for further processing in Python, where we will handle missing values and construct derived features.

### "BEDROOMS" and "BEDS" columns Missing value imputation

In [ ]:
from snowflake.snowpark import Session
session = Session.builder.configs(connection_parameters).create()

df = session.table("cleaned_listings").to_pandas()

for col in ['BEDROOMS', 'BEDS', 'PRICE']:
    df[col] = df.groupby(['NEIGHBOURHOOD_GROUP_CLEANSED', 'ROOM_TYPE'])[col]\
                .transform(lambda x: x.fillna(x.median()))

    df[col] = df[col].fillna(df[col].median())

df = df[~((df['BEDROOMS'] > 0) & (df['BEDS'] == 0))]

### Missing Data Imputation Summary
1. We impute bedrooms, beds, and price by grouping on NEIGHBOURHOOD_GROUP_CLEANSED and room_type, using the group-wise median, with a fallback to the global median when group data is insufficient.
2. We found 222 listings (3.32%) where bedrooms > 0 but beds == 0, which is logically inconsistent. These were likely due to data entry errors. To ensure data quality, we removed these records from the dataset.

### Transformation and Standardization

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # 检查价格分布
# sns.histplot(df['PRICE'], bins=100)
# plt.title("Price Distribution")
# plt.show()

# # 检查评论数分布
# sns.histplot(df['NUMBER_OF_REVIEWS'], bins=100)
# plt.title("Number of Reviews Distribution")
# plt.show()

# sns.histplot(df['REVIEWS_PER_MONTH'].dropna(), bins=50)
# plt.title("Distribution of NUMBER_OF_REVIEWS_PER_MONTH")
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# columns_to_check = ['ACCOMMODATES', 'BEDROOMS', 'BEDS', 'AVAILABILITY_365', 'HOST_LISTINGS_COUNT']

# for col in columns_to_check:
#     plt.figure(figsize=(6, 4))
#     sns.histplot(df[col].dropna(), bins=50, kde=False)
#     plt.title(f"Distribution of {col}")
#     plt.xlabel(col)
#     plt.ylabel("Frequency")
#     plt.show()


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. LOG_PRICE：处理价格偏态
df['LOG_PRICE'] = np.log1p(df['PRICE'])

# 2. LOG_REVIEWS_PER_MONTH：替代 NUMBER_OF_REVIEWS，处理偏态
df['LOG_REVIEWS_PER_MONTH'] = np.log1p(df['REVIEWS_PER_MONTH'])

# 3. LISTING_DENSITY：单位可出租天数上的活跃程度
df['LISTING_DENSITY'] = df['REVIEWS_PER_MONTH'] / (df['AVAILABILITY_365'] + 1)  # 避免除以 0

# 4. BEDROOM_BED_RATIO：结构合理性指标
df['BEDROOM_BED_RATIO'] = df['BEDROOMS'] / (df['BEDS'] + 1)  # 避免除以 0

# 必做
df['LOG_HOST_LISTINGS_COUNT'] = np.log1p(df['HOST_LISTINGS_COUNT'])

# 可选（建议保留做对比）
df['LOG_AVAILABILITY_365'] = np.log1p(df['AVAILABILITY_365'])

df = pd.get_dummies(df, columns=['ROOM_TYPE', 'PROPERTY_TYPE', 'NEIGHBOURHOOD_GROUP_CLEANSED'], drop_first=True)

# 选取用于建模的特征列
features = ['LOG_PRICE', 'ACCOMMODATES', 'BEDROOMS', 'BEDS', 'LOG_AVAILABILITY_365',
    'LOG_REVIEWS_PER_MONTH', 'LISTING_DENSITY', 'LOG_HOST_LISTINGS_COUNT', 'BEDROOM_BED_RATIO',
] + [col for col in df.columns if 'ROOM_TYPE_' in col or 'PROPERTY_TYPE_' in col]

X = df[features].copy()

# 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Feature Transformation Summary

In this section, we transformed key features to reduce skewness, normalize scales, and engineer meaningful ratios for downstream modeling.

#### Applied Transformations:
- **LOG_PRICE**: Applied `log1p(PRICE)` to reduce skewness in nightly price distribution.
- **LOG_REVIEWS_PER_MONTH**: Used `log1p(NUMBER_OF_REVIEWS_PER_MONTH)` instead of `NUMBER_OF_REVIEWS`, which contained only zeros.
- **LISTING_DENSITY**: Computed as `NUMBER_OF_REVIEWS_PER_MONTH / (AVAILABILITY_365 + 1)` to normalize review activity by listing availability.
- **BEDROOM_BED_RATIO**: Defined as `BEDROOMS / (BEDS + 1)` to detect configuration inconsistencies (e.g., listings with many bedrooms but few or no beds).
- **LOG_HOST_LISTINGS_COUNT**: Applied `log1p(HOST_LISTINGS_COUNT)` due to extreme right-skew in host portfolio sizes.
- **LOG_AVAILABILITY_365** (optional): Captures variation in listing availability, which showed a bimodal distribution (0 and 365 were common extremes).

#### No Transformation Needed:
- **ACCOMMODATES**, **BEDROOMS**, and **BEDS** are low-range discrete count features with relatively stable distributions, and were left untransformed.

These transformations help improve model interpretability, reduce the influence of extreme values, and support more effective clustering and anomaly detection.


In [ ]:
df.info()

In [ ]:
X

## Modeling

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

#可选：先降维到 10 维以内
pca = PCA(n_components=10)
X_reduced = pca.fit_transform(X_scaled)

# 运行 DBSCAN（参数可调试）
dbscan = DBSCAN(eps=1.5, min_samples=10)
df['CLUSTER_LABEL'] = dbscan.fit_predict(X_reduced)

In [ ]:
df

In [ ]:
print(df['CLUSTER_LABEL'].value_counts().sort_index())

In [ ]:
# 降到 2 维
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

# 添加到 df
df['PCA1'] = X_reduced[:, 0]
df['PCA2'] = X_reduced[:, 1]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
palette = sns.color_palette('tab20', n_colors=df['CLUSTER_LABEL'].nunique())

sns.scatterplot(
    data=df,
    x='PCA1',
    y='PCA2',
    hue='CLUSTER_LABEL',
    palette=palette,
    s=50,
    alpha=0.8,
    linewidth=0
)

plt.title('DBSCAN Clustering Result (PCA Projection)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Initial Clustering Summary

In this stage, we performed unsupervised clustering on Airbnb listings data to uncover natural groupings among listings:
- **Dimensionality Reduction**: We applied PCA to reduce the feature space to 10 dimensions to improve DBSCAN performance and enable visualization.
- **Clustering with DBSCAN**:
  - Used `DBSCAN` to discover density-based clusters without pre-specifying the number of clusters.
  - Identified multiple meaningful clusters along with a group of noise points (`CLUSTER_LABEL = -1`).
- **Visualization**: Plotted PCA-projected cluster assignments to evaluate separation quality and interpretability.
- **Goal**: This clustering provides a foundation for downstream **causal analysis**, allowing us to investigate **why listings fall into specific clusters** based on their attributes.


### Use causal inference to identify cluster label determinants

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 用你之前选择的特征（注意：全部大写）
features = ['LOG_PRICE', 'ACCOMMODATES', 'BEDROOMS', 'BEDS', 'LOG_AVAILABILITY_365',
    'LOG_REVIEWS_PER_MONTH', 'LISTING_DENSITY', 'LOG_HOST_LISTINGS_COUNT', 'BEDROOM_BED_RATIO',
] + [col for col in df.columns if 'ROOM_TYPE_' in col or 'PROPERTY_TYPE_' in col]

X = df[features]
y = df['CLUSTER_LABEL']

# 拆分训练测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

print("Train accuracy:", rf.score(X_train, y_train))
print("Test accuracy:", rf.score(X_test, y_test))

In [ ]:
import shap

# 创建 SHAP 解释器（TreeExplainer 专为树模型设计）
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

# 全局特征重要性图（支持多分类）
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# ---- 初始变量 ----
X_all = df[features].copy()
loop_history = []
max_loops = 5
early_stop_threshold = 0.01  # 当指标改善小于1%时停止
min_features = 3  # 最少要保留多少个特征

# 初始聚类标签
current_labels = df["CLUSTER_LABEL"].values

# ---- LOOP 开始 ----
for loop in range(max_loops):
    print(f"\n====== LOOP {loop+1} ======")

    # 1. 特征重要性分析（随机森林）
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_all, current_labels)
    importances = rf.feature_importances_
    importance_df = pd.DataFrame({
        'feature': X_all.columns,
        'importance': importances
    }).sort_values(by='importance', ascending=False)

    # 2. 选择前 N 个重要特征
    n_keep = max(min_features, int(len(X_all.columns) * 0.4))  # 只保留前40%重要的特征
    top_features = importance_df['feature'].iloc[:n_keep].tolist()
    X_selected = X_all[top_features]

    # 3. 标准化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)

    # 4. 聚类
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    new_labels = dbscan.fit_predict(X_scaled)

    # 如果聚不到簇或只聚出1类，跳出循环
    n_clusters = len(set(new_labels)) - (1 if -1 in new_labels else 0)
    if n_clusters <= 1:
        print("⚠️ 聚类簇数过少，跳出循环。")
        break

    # 5. 计算聚类评估指标
    sil_score = silhouette_score(X_scaled, new_labels)
    ch_score = calinski_harabasz_score(X_scaled, new_labels)
    db_score = davies_bouldin_score(X_scaled, new_labels)

    print(f"特征数量：{len(top_features)}")
    print(f"聚类数：{n_clusters}")
    print(f"Silhouette Score: {sil_score:.4f}")
    print(f"Calinski-Harabasz: {ch_score:.2f}")
    print(f"Davies-Bouldin: {db_score:.4f}")

    # 6. 保存历史结果
    loop_history.append({
        'loop': loop + 1,
        'features': top_features,
        'n_clusters': n_clusters,
        'silhouette': sil_score,
        'ch': ch_score,
        'db': db_score,
        'labels': new_labels
    })

    # 7. 判断是否 Early Stop
    if loop >= 1:
        prev = loop_history[-2]
        curr = loop_history[-1]
        sil_change = abs(curr['silhouette'] - prev['silhouette']) / (prev['silhouette'] + 1e-6)
        if sil_change < early_stop_threshold:
            print("✅ Silhouette 提升不足，Early Stop。")
            break

    # 8. 更新标签进入下一轮
    current_labels = new_labels
    X_all = df[top_features].copy()


In [ ]:
import matplotlib.pyplot as plt

sil_vals = [d['silhouette'] for d in loop_history]
plt.plot(range(1, len(sil_vals)+1), sil_vals, marker='o')
plt.title('Silhouette Score Across Loops')
plt.xlabel('Loop')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_all)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=current_labels, cmap='tab10', s=10, alpha=0.8)
plt.title("Final Cluster Visualization (t-SNE)")
plt.xlabel("t-SNE Dim 1")
plt.ylabel("t-SNE Dim 2")
plt.colorbar(scatter, label="Cluster Label")
plt.grid(True)
plt.show()


In [ ]:
df['FINAL_CLUSTER'] = loop_history[-1]['labels']
df['FINAL_CLUSTER'].value_counts()

In [ ]:
df.info()

## Anomaly detection

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter

def classify_listing_from_raw_input(user_input_raw, top_features, scaler, X_scaled, cluster_labels, rare_threshold=0.04):
    """
    Classify a new Airbnb listing from raw user input using final DBSCAN clustering.
    """

    # Step 1: Raw feature cleaning and engineering
    df_input = pd.DataFrame([user_input_raw])

    # Fill missing or derived-friendly defaults
    df_input["PRICE"] = df_input["PRICE"].astype(float)
    df_input["REVIEWS_PER_MONTH"] = df_input["REVIEWS_PER_MONTH"].astype(float)
    df_input["HOST_LISTINGS_COUNT"] = df_input["HOST_LISTINGS_COUNT"].astype(float)
    df_input["AVAILABILITY_365"] = df_input["AVAILABILITY_365"].astype(float)
    df_input["BEDROOMS"] = df_input["BEDROOMS"].fillna(1)
    df_input["BEDS"] = df_input["BEDS"].fillna(1)

    # Feature derivation
    df_input["LOG_PRICE"] = np.log1p(df_input["PRICE"])
    df_input["LOG_REVIEWS_PER_MONTH"] = np.log1p(df_input["REVIEWS_PER_MONTH"])
    df_input["LOG_HOST_LISTINGS_COUNT"] = np.log1p(df_input["HOST_LISTINGS_COUNT"])
    df_input["LOG_AVAILABILITY_365"] = np.log1p(df_input["AVAILABILITY_365"])
    df_input["LISTING_DENSITY"] = df_input["REVIEWS_PER_MONTH"] / (df_input["AVAILABILITY_365"] + 1)
    df_input["BEDROOM_BED_RATIO"] = df_input["BEDROOMS"] / (df_input["BEDS"] + 1)

    # Step 2: One-hot encode ROOM_TYPE and PROPERTY_TYPE
    room_dummies = pd.get_dummies(df_input["ROOM_TYPE"], prefix="ROOM_TYPE")
    prop_dummies = pd.get_dummies(df_input["PROPERTY_TYPE"], prefix="PROPERTY_TYPE")
    df_input = pd.concat([df_input, room_dummies, prop_dummies], axis=1)

    # Step 3: Match top_features
    for col in top_features:
        if col not in df_input.columns:
            df_input[col] = 0  # Fill missing one-hot columns with 0

    X_user = df_input[top_features]
    X_user_scaled = scaler.transform(X_user)

    # Step 4: Compare to cluster centers
    cluster_sizes = Counter(cluster_labels)
    cluster_centers = {}
    cluster_distances = {}

    for label in set(cluster_labels):
        if label == -1:
            continue
        cluster_points = X_scaled[cluster_labels == label]
        center = cluster_points.mean(axis=0)
        cluster_centers[label] = center
        cluster_distances[label] = euclidean_distances(X_user_scaled, center.reshape(1, -1))[0][0]

    if not cluster_distances:
        return {"type": "anomaly", "reason": "No clusters available."}

    closest_cluster = min(cluster_distances, key=cluster_distances.get)
    closest_distance = cluster_distances[closest_cluster]
    cluster_ratio = cluster_sizes[closest_cluster] / len(cluster_labels)

    # 95% cutoff in this cluster
    own_distances = euclidean_distances(X_scaled[cluster_labels == closest_cluster], cluster_centers[closest_cluster].reshape(1, -1))
    abnormal_cutoff = np.percentile(own_distances, 95)

    if closest_distance > abnormal_cutoff:
        label_type = "anomaly"
    elif cluster_ratio < rare_threshold:
        label_type = "rare"
    else:
        label_type = "typical"

    return {
        "type": label_type,
        "closest_cluster": int(closest_cluster),
        "cluster_size_ratio": round(cluster_ratio, 4),
        "distance_to_cluster_center": round(closest_distance, 4),
        "abnormal_cutoff": round(abnormal_cutoff, 4)
    }


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_anomaly_detector(test_listings, classify_func, top_features, scaler, X_scaled, cluster_labels):
    """
    test_listings: list of dicts, each with input fields + LABEL
    classify_func: function that outputs {type: "typical" / "rare" / "anomaly"}
    top_features, scaler, X_scaled, cluster_labels: as used in classification
    """

    y_true = []
    y_pred = []

    for listing in test_listings:
        label = listing.pop("LABEL")  # extract true label
        y_true.append(label)

        try:
            result = classify_func(
                user_input_raw=listing,
                top_features=top_features,
                scaler=scaler,
                X_scaled=X_scaled,
                cluster_labels=cluster_labels
            )
            y_pred.append(result["type"])
        except Exception as e:
            y_pred.append("error")

    # Report
    labels_order = ["typical", "rare", "anomaly"]
    cm = confusion_matrix(y_true, y_pred, labels=labels_order)
    report = classification_report(y_true, y_pred, labels=labels_order, output_dict=True)

    return {
        "confusion_matrix": pd.DataFrame(cm, index=labels_order, columns=labels_order),
        "classification_report": pd.DataFrame(report).T,
        "y_true": y_true,
        "y_pred": y_pred
    }


In [ ]:
# Generate 100 labeled test listings: 70 typical, 15 rare, 15 anomaly
full_test_listings = []

# Generate 70 typical
for _ in range(350):
    full_test_listings.append({
        "ROOM_TYPE": "Entire home/apt",
        "PROPERTY_TYPE": "Apartment",
        "ACCOMMODATES": np.random.randint(2, 5),
        "BEDROOMS": np.random.randint(1, 3),
        "BEDS": np.random.randint(1, 4),
        "PRICE": np.random.randint(80, 180),
        "AVAILABILITY_365": np.random.randint(150, 365),
        "HOST_LISTINGS_COUNT": np.random.randint(1, 3),
        "REVIEWS_PER_MONTH": np.random.normal(1.0, 0.5),
        "LABEL": "typical"
    })

# Generate 15 rare
for _ in range(75):
    full_test_listings.append({
        "ROOM_TYPE": "Shared room",
        "PROPERTY_TYPE": "Hostel",
        "ACCOMMODATES": np.random.randint(3, 6),
        "BEDROOMS": np.random.randint(1, 3),
        "BEDS": np.random.randint(3, 5),
        "PRICE": np.random.randint(20, 50),
        "AVAILABILITY_365": 365,
        "HOST_LISTINGS_COUNT": 1,
        "REVIEWS_PER_MONTH": np.random.normal(0.5, 0.2),
        "LABEL": "rare"
    })

# Generate 15 anomaly
for _ in range(150):
    full_test_listings.append({
        "ROOM_TYPE": "Private room",
        "PROPERTY_TYPE": "Condominium",
        "ACCOMMODATES": np.random.randint(20, 30),
        "BEDROOMS": 9,
        "BEDS": 1,
        "PRICE": 2000,
        "AVAILABILITY_365": 30,
        "HOST_LISTINGS_COUNT": np.random.randint(20, 50),
        "REVIEWS_PER_MONTH": np.random.normal(0.1, 0.1),
        "LABEL": "anomaly"
    })

# # Generate 70 typical
# for _ in range(70):
#     full_test_listings.append({
#         "ROOM_TYPE": "Entire home/apt",
#         "PROPERTY_TYPE": "Apartment",
#         "ACCOMMODATES": np.random.randint(2, 6),
#         "BEDROOMS": np.random.randint(1, 3),
#         "BEDS": np.random.randint(1, 4),
#         "PRICE": np.random.randint(80, 180),
#         "AVAILABILITY_365": np.random.randint(150, 380),
#         "HOST_LISTINGS_COUNT": np.random.randint(1, 3),
#         "REVIEWS_PER_MONTH": np.random.normal(1.0, 0.6),
#         "LABEL": "typical"
#     })

# # Generate 15 rare
# for _ in range(15):
#     full_test_listings.append({
#         "ROOM_TYPE": "Shared room",
#         "PROPERTY_TYPE": "Hostel",
#         "ACCOMMODATES": np.random.randint(3, 5),
#         "BEDROOMS": np.random.randint(1, 3),
#         "BEDS": np.random.randint(4, 5),
#         "PRICE": np.random.randint(20, 50),
#         "AVAILABILITY_365": 360,
#         "HOST_LISTINGS_COUNT": 1,
#         "REVIEWS_PER_MONTH": np.random.normal(0.6, 0.2),
#         "LABEL": "rare"
#     })

# # Generate 15 anomaly
# for _ in range(15):
#     full_test_listings.append({
#         "ROOM_TYPE": "Private room",
#         "PROPERTY_TYPE": "Condominium",
#         "ACCOMMODATES": np.random.randint(1, 2),
#         "BEDROOMS": np.random.randint(4, 10),
#         "BEDS": np.random.randint(1, 2),
#         "PRICE": np.random.randint(500, 2000),
#         "AVAILABILITY_365": np.random.randint(1, 30),
#         "HOST_LISTINGS_COUNT": np.random.randint(20, 50),
#         "REVIEWS_PER_MONTH": np.random.normal(0.1, 0.1),
#         "LABEL": "anomaly"
#     })

# Shuffle the list
np.random.shuffle(full_test_listings)

In [ ]:
result = evaluate_anomaly_detector(
    test_listings=full_test_listings,             # 你刚才生成的100条数据
    classify_func=classify_listing_from_raw_input,  # 判定函数
    top_features=top_features,                    # 你的最终特征名列表
    scaler=scaler,                                # 标准化器
    X_scaled=X_scaled,                            # 训练数据（已标准化）
    cluster_labels=new_labels                     # DBSCAN 输出标签
)
result["confusion_matrix"]         # 混淆矩阵（DataFrame）
result["classification_report"]   # precision / recall / f1-score
result["y_true"], result["y_pred"] # 原始标签与预测

In [ ]:
import joblib, json, numpy as np

# 1) 模型工件
joblib.dump(scaler, 'scaler.pkl')
with open('top_features.json', 'w') as f:
    json.dump(top_features, f)
np.save('X_scaled.npy', X_scaled)
np.save('cluster_labels.npy', np.array(new_labels))

In [ ]:
import os

# 列出当前目录所有文件
print(os.listdir("."))

# 如果要下载某个文件到本地
import snowflake.snowpark as snowpark  # 如果支持
# 或者直接 Notebook 提供的 download 工具按钮


In [ ]:
import zipfile

# 把要下载的文件都加进来
with zipfile.ZipFile("artifacts.zip", "w") as zf:
    zf.write("scaler.pkl")
    zf.write("top_features.json")
    zf.write("X_scaled.npy")
    zf.write("cluster_labels.npy")
    # 如果你有 full_test_listings.json 也可以加进来
    # zf.write("full_test_listings.json")

print("打包完成: artifacts.zip")

In [ ]:
import base64
from IPython.display import HTML, display

def create_download_link(filename):
    with open(filename, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()
    return HTML(f'<a href="data:application/zip;base64,{b64}" download="{filename}">下载 {filename}</a>')

display(create_download_link("artifacts.zip"))

